## 001 データを読み込んでみる

In [ ]:
import datetime
import pandas as pd

In [ ]:
cutomer_master = pd.read_csv('./input/100knoks/01-01/customer_master.csv')
cutomer_master.head()

In [ ]:
item_master = pd.read_csv('./input/100knoks/01-01/item_master.csv')
item_master.head()

In [ ]:
transaction_1 = pd.read_csv('./input/100knoks/01-01/transaction_1.csv')
transaction_1.head()

In [ ]:
transaction_2 = pd.read_csv('./input/100knoks/01-01/transaction_2.csv')
transaction_2.head()

In [ ]:
transaction_detail_1 = pd.read_csv('./input/100knoks/01-01/transaction_detail_1.csv')
transaction_detail_1.head()

In [ ]:
transaction_detail_2 = pd.read_csv('./input/100knoks/01-01/transaction_detail_2.csv')
transaction_detail_2.head()

## 002 データを結合してみる Union

In [ ]:
transaction = pd.concat([transaction_1, transaction_2], ignore_index=True)
transaction.head()

In [ ]:
print(len(transaction_1))
print(len(transaction_2))
print(len(transaction))


In [ ]:
transaction_detail = pd.concat([transaction_detail_1, transaction_detail_2], ignore_index=True)
transaction_detail.head()

In [ ]:
print(len(transaction_detail_1))
print(len(transaction_detail_2))
print(len(transaction_detail))

## 003 データを結合する Join

In [ ]:
transaction_join = pd.merge(transaction_detail, transaction[['transaction_id', 'payment_date', 'customer_id']], on='transaction_id', how='left')
transaction_join.head()

In [ ]:
print(len(transaction_detail))
print(len(transaction))
print(len(transaction_join))

## 004 マスターを結合する Join

In [ ]:
#transaction_join = pd.merge(transaction_join, cutomer_master[['customer_id', 'customer_name', 'registration_date', 'customer_name_kana', 'email', 'gender', 'age', 'birth', 'pref']], on='customer_id', how='left')
transaction_join = pd.merge(transaction_join, cutomer_master, on='customer_id', how='left')
transaction_join = pd.merge(transaction_join, item_master, on='item_id', how='left')
transaction_join.head()

In [ ]:
print(len(transaction))
print(len(transaction_detail))
print(len(transaction_join))

## 005 売上の列を追加

In [ ]:
transaction_join['price'] = transaction_join['quantity'] * transaction_join['item_price']
transaction_join[['quantity', 'item_price', 'price']].head()

## 006 データを検算
合計値があってるかを比較して検算

In [ ]:
print(transaction_join['price'].sum() == transaction['price'].sum())
print(transaction_join['price'].sum())
print(transaction['price'].sum())

## 007 統計量の把握

In [ ]:
transaction_join.describe()

In [ ]:
transaction_join.isnull().sum()

## 008 月別に集計

In [ ]:
transaction_join.dtypes

In [ ]:
transaction_join['patment_date'] = pd.to_datetime(transaction_join['payment_date'])
transaction_join['payment_month'] = transaction_join['patment_date'].dt.strftime("%Y%m")
transaction_join[['payment_date', 'payment_month']].head()

In [ ]:
transaction_join.groupby('payment_month').sum()['price']

## 009 月別、商品別で集計

groupbyを使って表示

In [ ]:
transaction_join.groupby(['payment_month', 'item_name']).sum()[['price', 'quantity']]

Pivotを使って表示

In [ ]:
pd.pivot_table(transaction_join, index='item_name', columns='payment_month', values=['price', 'quantity'], aggfunc='sum')

## 010 可視化してみる

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
graph_data = transaction_join.groupby(['item_name', 'payment_month']).sum()
sns.set()
sns.lineplot(x='payment_month', y='price', hue='item_name', data=graph_data)
